# 第7章 数据清洗和准备

在数据分析和建模的过程中，相当多的时间要用在数据准备上：加载、清理、转换以及重塑。这些工作会占到分析师时间的80%或更多。有时，存储在文件和数据库中的数据的格式不适合某个特定的任务。许多研究者都选择使用通用编程语言（如Python、Perl、R或Java）或UNIX文本处理工具（如sed或awk）对数据格式进行专门处理。幸运的是，pandas和内置的Python标准库提供了一组高级的、灵活的、快速的工具，可以让你轻松地将数据规整为想要的格式。

In [1]:
import numpy as np
import pandas as pd

## 7.1 处理缺失数据

pandas使用浮点值NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以方便的检测出来：

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

采用了R语言中的惯用法，即将缺失值表示为NA，它表示不可用not available。在统计应用中，NA数据可能是不存在的数据或者虽然存在，但是没有观察到（例如，数据采集中发生了问题）。当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

dropna:根据是否缺失数据对轴标签进行过滤，可通过阀值调整对缺失值的容忍度；
fillna：用指定值或插值方法（ffill或bfill）填充确实数据；
isnull：返回一个含有布尔值的对象；
notnull:isnull的否定式

## 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布尔索引的手工方法，但dropna可能会更实用一些。对于一个Series，dropna返回一个仅含非空数据和索引值的Series：

In [8]:
from numpy import nan as NA

In [9]:
data = pd.Series([1,NA,3.5,NA,7])

In [10]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NA或含有NA的行或列。dropna默认丢弃任何含有缺失值的行：

In [18]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],[NA, NA, NA], [NA, 6.5, 3.]])

In [12]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
cleaned = data.dropna()

In [20]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [21]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [22]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [23]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


### 另一个滤除DataFrame行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用thresh参数实现此目的：

In [24]:
df = pd.DataFrame(np.random.randn(7, 3))

In [25]:
df.iloc[:4, 1] = NA

In [26]:
df.iloc[:2, 2] = NA

In [27]:
df

,0,1,2
0,-1.572747,NaN,NaN
1,1.454101,NaN,NaN
2,-1.675355,NaN,-0.011808
3,0.775440,NaN,-1.632290
4,1.138064,-0.523005,-0.107557
5,1.657226,-1.666499,-0.757678
6,-1.124599,-0.828739,1.413721


In [28]:
df.dropna()

,0,1,2
4,1.138064,-0.523005,-0.107557
5,1.657226,-1.666499,-0.757678
6,-1.124599,-0.828739,1.413721


In [29]:
df.dropna(thresh=2)

,0,1,2
2,-1.675355,NaN,-0.011808
3,0.775440,NaN,-1.632290
4,1.138064,-0.523005,-0.107557
5,1.657226,-1.666499,-0.757678
6,-1.124599,-0.828739,1.413721


## 填充缺失数据

你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。对于大多数情况而言，fillna方法是最主要的函数。通过一个常数调用fillna就会将缺失值替换为那个常数值：

In [30]:
df.fillna(0)

,0,1,2
0,-1.572747,0.000000,0.000000
1,1.454101,0.000000,0.000000
2,-1.675355,0.000000,-0.011808
3,0.775440,0.000000,-1.632290
4,1.138064,-0.523005,-0.107557
5,1.657226,-1.666499,-0.757678
6,-1.124599,-0.828739,1.413721


### 若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：

In [31]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-1.572747,0.500000,0.000000
1,1.454101,0.500000,0.000000
2,-1.675355,0.500000,-0.011808
3,0.775440,0.500000,-1.632290
4,1.138064,-0.523005,-0.107557
5,1.657226,-1.666499,-0.757678
6,-1.124599,-0.828739,1.413721


fillna默认会返回新对象，但也可以对现有对象进行就地修改：

In [32]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-1.572747,0.000000,0.000000
1,1.454101,0.000000,0.000000
2,-1.675355,0.000000,-0.011808
3,0.775440,0.000000,-1.632290
4,1.138064,-0.523005,-0.107557
5,1.657226,-1.666499,-0.757678
6,-1.124599,-0.828739,1.413721


对reindexing有效的那些插值方法也可用于fillna：

In [33]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.085538,-1.235117,-1.302707
1,-0.547585,-1.265146,-1.782321
2,0.748365,NaN,0.436143
3,0.638596,NaN,0.042743
4,1.485621,NaN,NaN
5,0.157606,NaN,NaN


In [34]:
df.fillna(method='ffill')

,0,1,2
0,0.085538,-1.235117,-1.302707
1,-0.547585,-1.265146,-1.782321
2,0.748365,-1.265146,0.436143
3,0.638596,-1.265146,0.042743
4,1.485621,-1.265146,0.042743
5,0.157606,-1.265146,0.042743


In [35]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.085538,-1.235117,-1.302707
1,-0.547585,-1.265146,-1.782321
2,0.748365,-1.265146,0.436143
3,0.638596,-1.265146,0.042743
4,1.485621,NaN,0.042743
5,0.157606,NaN,0.042743


In [36]:
df.fillna(df.mean())

,0,1,2
0,0.085538,-1.235117,-1.302707
1,-0.547585,-1.265146,-1.782321
2,0.748365,-1.250131,0.436143
3,0.638596,-1.250131,0.042743
4,1.485621,-1.250131,-0.651536
5,0.157606,-1.250131,-0.651536


## 7.2 数据转换

### 移除重复数据,DataFrame中出现重复行有多种原因。下面就是一个例子：

In [37]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],'k2': [1, 1, 2, 3, 3, 4, 4]})

In [38]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [39]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [40]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [44]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [45]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [46]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 利用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：.str.lower(),.map()

In [47]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon','Pastrami', 'corned beef', 'Bacon','pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

In [48]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

Series的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。因此，我们还需要使用Series的str.lower方法，将各个值转换为小写：

In [49]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [50]:
data['animal'] = lowercased.map(meat_to_animal)

In [51]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [52]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 替换值--map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式。

In [53]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [54]:
data.replace(-999, np.nan)#-999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值，

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：：

In [55]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

### 重命名轴索引

### 离散化和面元划分

### 检测和过滤异常值

### 排列和随机采样

### 计算指标/哑变量

## 7.3 字符串操作

### 字符串对象方法

### 正则表达式

### pandas的矢量化字符串函数